# Finding out spam and ham emails with the help of RNN(Recurrent Neural Network)

# Author: Rohan Kulkarni

In [1]:
import pyforest

In [2]:
spam=pd.read_csv(r'E:\Imarticus\14.NATURAL LANGUAGE PROCESSING\spam1.csv',encoding='cp1252')

<IPython.core.display.Javascript object>

In [4]:
spam.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
spam.v1.value_counts()

ham     5854
spam     922
Name: v1, dtype: int64

In [8]:
spam.v1.replace({'ham':0,'spam':1},inplace=True)

In [9]:
spam.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [12]:
spam.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True,axis=1)

In [13]:
spam.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
from sklearn.model_selection import train_test_split



In [15]:
x_spam=spam.v2
y_spam=spam.v1

In [26]:
x_train,x_test,y_train,y_test=train_test_split(x_spam,y_spam,test_size=.2,random_state=10)

In [27]:
x_train.shape

(5420,)

In [28]:
x_test.shape

(1356,)

In [29]:
x_spam.shape

(6776,)

In [30]:
y_train.shape

(5420,)

In [31]:
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import to_categorical

In [34]:
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [36]:
max_num=10000
seq_len=100
embedding_size=100

In [37]:
from keras.preprocessing.text import Tokenizer # is used for tokenization
from keras.preprocessing.sequence import pad_sequences

In [38]:
tokenizer=Tokenizer(num_words=max_num)  #
tokenizer.fit_on_texts(spam.v2)         # would be used----It is tokenizing entire amazon.review             
x_train=tokenizer.texts_to_sequences(x_train) # and text_to_sequences mapping it from reviews to only train
                                              #will conert the text to sequence of IDs
x_train=pad_sequences(x_train,maxlen=seq_len) 

In [39]:
x_test=tokenizer.texts_to_sequences(x_test)
x_test=pad_sequences(x_test,maxlen=seq_len) 

In [40]:
from tensorflow.keras.optimizers import Adam

from keras.layers import Dense,LSTM,Embedding
from keras.models import Model,Sequential

In [41]:
model=Sequential()  # initialize the network
model.add(Embedding(input_dim=max_num,
                    input_length=seq_len,
                    output_dim=embedding_size))

model.add(LSTM(50))
model.add(Dense(2,activation='softmax'))

adam=Adam(lr=.01)

C:\Users\_RoXstaR_\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [42]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [43]:
model.fit(x_train,y_train,epochs=7,batch_size=50)

Epoch 1/7
109/109 [==============================] - 5s 32ms/step - loss: 0.2368 - accuracy: 0.9277
Epoch 2/7
109/109 [==============================] - 3s 32ms/step - loss: 0.0319 - accuracy: 0.9935 0s - loss: 0.0
Epoch 3/7
109/109 [==============================] - 3s 29ms/step - loss: 0.0129 - accuracy: 0.9974
Epoch 4/7
109/109 [==============================] - 3s 29ms/step - loss: 0.0050 - accuracy: 0.9991
Epoch 5/7
109/109 [==============================] - 3s 30ms/step - loss: 0.0021 - accuracy: 0.9998
Epoch 6/7
109/109 [==============================] - 3s 30ms/step - loss: 9.9736e-04 - accuracy: 0.9998
Epoch 7/7
109/109 [==============================] - 3s 30ms/step - loss: 8.6553e-04 - accuracy: 0.9998


In [44]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [45]:
pred=model.predict(x_test)

In [46]:
pred_argmax=np.argmax(pred,axis=1)

<IPython.core.display.Javascript object>

In [47]:
confusion_matrix(pred_argmax,y_test[:,1])

array([[1183,    9],
       [   4,  160]], dtype=int64)

In [48]:
accuracy_score(y_test[:,1],pred_argmax)

0.9904129793510325